In [1]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn

In [2]:
folder = r"../lib/total_data"
file_names = os.listdir(folder)
folder_30s = r"../lib/attention_model_data/30s"
folder_28s = r"../lib/attention_model_data/28s"
folder_26s = r"../lib/attention_model_data/26s"

hard_landing_threshold = 1.6
heavy_landing_threshold = 1.9

columns_to_extract = [5, 8, 11, 12, 13, 14, 15, 16, 19, 26, 28, 30, 32, 33, 34, 35, 36, 37, 41, 42, 43, 44, 45]
column_names = ['FLIGHT PHASE', 'DIST TO PSEUDO THRESHOLD', 'COMPUTED AIRSPEED', 'VERTICAL SPEED',
                'VERTICAL ACCELERATION ACQ', 'CAP DISP HEADING', 'LOCALIZER DEV IN DOTS',
                'GLIDE DEVIATION IN DOTS', 'FLIGHT PATH ANGLE', 'RADIO ALTITUDE', 'PITCH ANGLE',
                'ROLL ANGLE', 'N11', 'N22', 'SELTD N1 INDICATED 1', 'SELTD N1 INDICATED 2',
                'WIND SPEED', 'WIND DIRECTION', 'CONTROL COLUMN POSITION',
                'CONTROL WHEEL POSITION', 'RUDDER PEDAL', 'THROTTLE ANGLE1', 'THROTTLE ANGLE2',]

In [3]:
# 数据选择 选择垂直加速度最大的点为时间点t，分别选择[t-30,t],[t-30,t-2],[t-30,t-4]三个时间段
# 参考的论文:<A Dual Two-Stage Attention-based Model for interpretable hard landing prediction from flight data>
target_y = []
target_x_30s = []
target_x_28s = []
target_x_26s = []

for f in file_names:
    # 读文件
    file = os.path.join(folder, f)
    df = pd.read_csv(file, skiprows=[0, 1, 3, 4], header=0)
    df = df.iloc[:,columns_to_extract]
    df.columns = column_names
    
    #分割时间片段
    max_g_index =  df['VERTICAL ACCELERATION ACQ'].idxmax()
    df_30s = df.iloc[max_g_index - 30: max_g_index]
    df_28s = df.iloc[max_g_index - 30: max_g_index - 2]
    df_26s = df.iloc[max_g_index - 30: max_g_index - 4]

    #选择训练所使用的特征
    selected_column_names = ['DIST TO PSEUDO THRESHOLD', 'COMPUTED AIRSPEED', 'VERTICAL SPEED', 'CAP DISP HEADING', 'LOCALIZER DEV IN DOTS',
                             'GLIDE DEVIATION IN DOTS', 'FLIGHT PATH ANGLE', 'RADIO ALTITUDE', 'PITCH ANGLE','ROLL ANGLE', 'N11', 'N22',
                             'SELTD N1 INDICATED 1', 'SELTD N1 INDICATED 2','WIND SPEED', 'WIND DIRECTION', 'CONTROL COLUMN POSITION',
                             'CONTROL WHEEL POSITION', 'RUDDER PEDAL', 'THROTTLE ANGLE1', 'THROTTLE ANGLE2',]
    df_30s = df_30s[selected_column_names]
    df_30s.reset_index(drop=True, inplace=True)
    
    df_28s = df_28s[selected_column_names]
    df_28s.reset_index(drop=True, inplace=True)
    
    df_26s = df_26s[selected_column_names]
    df_26s.reset_index(drop=True, inplace=True)
    
    # 给样本打上标签
    max_g = df['VERTICAL ACCELERATION ACQ'].iloc[max_g_index]
    if max_g < 1:
        print("最大g值小于1，可能发生错误")
        print("max_g:", max_g)
    if max_g < hard_landing_threshold and max_g > 1:
        category = 0
    elif max_g < heavy_landing_threshold:
        category = 1
    else:
        category = 2
        
    # 暂存数据
    target_y.append(category)
    
    res_x_30s = os.path.join(folder_30s, f)
    df_30s.to_csv(res_x_30s, index=False)
    
    res_x_28s = os.path.join(folder_28s, f)
    df_28s.to_csv(res_x_28s, index=False)
    
    res_x_26s = os.path.join(folder_26s, f)
    df_26s.to_csv(res_x_30s, index=False)


C:\Users\Szoke\AppData\Local\Temp\ipykernel_5952\378309212.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=[0, 1, 3, 4], header=0)
C:\Users\Szoke\AppData\Local\Temp\ipykernel_5952\378309212.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=[0, 1, 3, 4], header=0)
C:\Users\Szoke\AppData\Local\Temp\ipykernel_5952\378309212.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=[0, 1, 3, 4], header=0)
C:\Users\Szoke\AppData\Local\Temp\ipykernel_5952\378309212.py:11: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, skiprows=[0, 1, 3, 4], header=0)
C:\Users\Szoke\AppData\Local\Temp\ipykernel_5952\378309212.py:11: DtypeWarning: Columns (6) have mixed t